In [ ]:
# External imports
import numpy as np
import pandas as pd
import torch
from typing import Tuple, List, Dict

# Project imports
from squad_data.parser import SquadFileParser
from squad_data.utils import build_mappers_and_dataframe

In [ ]:
### Download Embedding

In [ ]:
from utils.embedding_utils import EmbeddingDownloader

embedding_downloader = EmbeddingDownloader(
    "embedding_models", 
    "embedding_model.kv", 
    model_name="fasttext-wiki-news-subwords-300"
)

embedding_model = embedding_downloader.load()

In [ ]:
### Parse the json and get the data

In [ ]:
parser = SquadFileParser("squad_data/data/training_set.json")
data = parser.parse_documents()

In [ ]:
### Prepare the mappers and dataframe

In [ ]:
paragraphs_mapper, questions_mapper, df = build_mappers_and_dataframe(data)
print(questions_mapper[next(iter(questions_mapper))])
print(paragraphs_mapper[next(iter(paragraphs_mapper))])
df.head()

In [ ]:
# Extend the paragraphs mapper to include spans
paragraphs_spans_mapper = add_paragraphs_spans(paragraphs_mapper)

In [ ]:
print(paragraphs_spans_mapper['0_0']['text'])
print(paragraphs_spans_mapper['0_0']['spans']

In [ ]:
### DataConverter and CustomQADataset

In [ ]:
from data_loading.utils import DataConverter, padder_collate_fn
from data_loading.qa_dataset import CustomQADataset

data_converter = DataConverter(embedding_model, paragraphs_spans_mapper)
datasetQA = CustomQADataset(data_converter, df, paragraphs_mapper, questions_mapper)
data_loader = torch.utils.data.DataLoader(datasetQA, collate_fn = padder_collate_fn, batch_size=10, shuffle=True)
print(next(iter(data_loader))[0].shape)
print(next(iter(data_loader))[2].shape)